In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import pyspark.sql.types as t

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_companies_task") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_movies = spark.read.option("header","true").csv("./movies_metadata.csv")
df_movie_prod_companies = df_movies.select(f.col("id").alias("movie_tmdb_id"), f.col("production_companies"))
df_movie_prod_companies = df_movie_prod_companies.withColumn("production_companies", f.from_json(f.col("production_companies"), "array<string>"))
df_movie_prod_companies = df_movie_prod_companies.na.drop(subset=['production_companies'])
df_movie_prod_companies = df_movie_prod_companies.withColumn("production_companies", f.explode("production_companies"))
df_movie_prod_companies.printSchema()
df_movie_prod_companies.show(10, truncate=False)

In [ ]:
schema = t.StructType(
    [
        t.StructField('id', t.StringType(), True),
        t.StructField('name', t.StringType(), True)
    ]
)
df_movie_prod_companies = df_movie_prod_companies.withColumn('production_companies', f.from_json('production_companies', schema))\
.select(f.col('movie_tmdb_id'), f.col('production_companies.id').alias('company_id'), f.col('production_companies.name'))
df_movie_prod_companies = df_movie_prod_companies.na.drop('any')
df_movie_prod_companies.printSchema()
df_movie_prod_companies.show(10, truncate=False)

In [ ]:
df_movie_prod_companies.select(*df_movie_prod_companies.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver", "com.mysql.cj.jdbc.Driver").option("dbtable", "movie_prod_companies") \
    .option("user", "root").option("password", "zhuyz928").save()

In [ ]:
df_prod_companies = df_movie_prod_companies.select(['company_id', 'name']).distinct()
df_prod_companies.sort(f.col('company_id'))

In [ ]:
df_prod_companies.select(*df_prod_companies.columns).write.format("jdbc") \
    .option("url", "jdbc:mysql://localhost:3306/spark_movies") \
    .option("driver","com.mysql.cj.jdbc.Driver").option("dbtable", "prod_companies") \
    .option("user", "root").option("password", "zhuyz928").save()